In [25]:
#As seen from the results below, this table is unwieldy and difficult to use for our requirments. 
#Therefore, this script/notebook contains scripts to convert the source data into a form we like and as described in our reports.
#The data we get from this notebook is used to populate our database

In [26]:
import pandas as pd


In [27]:
df = pd.read_csv('MCI_2014_to_2019.csv')
btDF = pd.read_csv('Bicycle_Thefts.csv')
pdDF = pd.read_csv('tablepoliceDivision.csv')

In [28]:
print(df.columns.tolist())
print(btDF.columns.tolist())
print(pdDF.columns.tolist())

['X', 'Y', 'Index_', 'event_unique_id', 'occurrencedate', 'reporteddate', 'premisetype', 'ucr_code', 'ucr_ext', 'offence', 'reportedyear', 'reportedmonth', 'reportedday', 'reporteddayofyear', 'reporteddayofweek', 'reportedhour', 'occurrenceyear', 'occurrencemonth', 'occurrenceday', 'occurrencedayofyear', 'occurrencedayofweek', 'occurrencehour', 'MCI', 'Division', 'Hood_ID', 'Neighbourhood', 'Long', 'Lat', 'ObjectId']
['X', 'Y', 'FID', 'Index_', 'event_unique_id', 'Primary_Offence', 'Occurrence_Date', 'Occurrence_Year', 'Occurrence_Month', 'Occurrence_Day', 'Occurrence_Time', 'Division', 'City', 'Location_Type', 'Premise_Type', 'Bike_Make', 'Bike_Model', 'Bike_Type', 'Bike_Speed', 'Bike_Colour', 'Cost_of_Bike', 'Status', 'Hood_ID', 'Neighbourhood', 'Lat', 'Long']
['division', 'address', 'area', 'shapeLeng', 'shapeArea']


In [29]:
df2_1 = df[[ "occurrencehour", "occurrenceday","occurrencemonth", "occurrenceyear", "occurrencedayofweek"]]
df2_2 = df[["reportedhour", "reportedday","reportedmonth", "reportedyear", "reporteddayofweek"]]
df2_1.rename(columns={"occurrencehour":"hour", "occurrenceday":"day", "occurrencemonth":"month", "occurrenceyear":"year", "occurrencedayofweek":"day_of_week"}, inplace=True)
df2_2.rename(columns={"reportedhour":"hour", "reportedday":"day", "reportedmonth":"month", "reportedyear":"year", "reporteddayofweek":"day_of_week"}, inplace=True)
df2 = df2_1.append(df2_2)
df2 = df2.drop_duplicates()
df2 = df2.rename_axis('time_id').reset_index()
df2['time_id'] += 600000
df2.to_csv("IncidentTime.csv", index=False)
print(df2)

/home/dragonemperor/.local/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


       time_id  hour   day      month    year day_of_week
0       600000     1  24.0      March  2014.0  Monday    
1       600001    16  27.0  September  2014.0  Saturday  
2       600002     6  24.0      March  2014.0  Monday    
3       600003    15  24.0      March  2014.0  Monday    
4       600004     2   3.0        May  2014.0  Saturday  
...        ...   ...   ...        ...     ...         ...
52030   805289     5  27.0      April  2018.0  Friday    
52031   805290     7  27.0      April  2018.0  Friday    
52032   805291     6  27.0      April  2018.0  Friday    
52033   805949     6  21.0      March  2019.0  Thursday  
52034   806309     6   8.0     August  2019.0  Thursday  

[52035 rows x 6 columns]


In [30]:
df3 = df[["Hood_ID", "Neighbourhood", "Division"]]
df3 = df3.drop_duplicates()
df3 = df3.rename_axis('locationID').reset_index()
df3['locationID'] += 300000

print(df3)

     locationID  Hood_ID                     Neighbourhood Division
0        300000      132                     Malvern (132)      D42
1        300001       76          Bay Street Corridor (76)      D52
2        300002        1        West Humber-Clairville (1)      D23
3        300003       47           Don Valley Village (47)      D33
4        300004       90                Junction Area (90)      D11
..          ...      ...                               ...      ...
224      404110      109          Caledonia-Fairbank (109)      D12
225      405320        3  Thistletown-Beaumond Heights (3)      D31
226      416423       44              Flemingdon Park (44)      D33
227      421705       75        Church-Yonge Corridor (75)      D12
228      427148       32          Englemount-Lawrence (32)      D42

[229 rows x 4 columns]


In [31]:
df4 = df[[ "offence", "MCI"]]
df4 = df4.drop_duplicates()
df4 = df4.rename_axis('crimeID').reset_index()
df4['crimeID'] += 100000
df4.to_csv("tableCrime.csv", index=False)
print(df4)

    crimeID                         offence              MCI
0    100000                         Assault          Assault
1    100001                             B&E  Break and Enter
2    100004              Robbery - Business          Robbery
3    100005             Assault With Weapon          Assault
4    100008                      Theft Over       Theft Over
5    100009               Robbery - Mugging          Robbery
6    100011        Theft Over - Shoplifting       Theft Over
7    100026                    B&E W'Intent  Break and Enter
8    100035              Aggravated Assault          Assault
9    100040             Assault Bodily Harm          Assault
10   100042  Assault - Resist/ Prevent Seiz          Assault
11   100056                 Robbery - Other          Robbery
12   100061             Robbery With Weapon          Robbery
13   100065           Assault Peace Officer          Assault
14   100072         Robbery - Home Invasion          Robbery
15   100091    Unlawfull

In [32]:
df5 = df.merge(df4, on=['offence','MCI'] )
df6 = df5.merge(df3, on=["Hood_ID", "Neighbourhood"])
df6.rename(columns={"occurrencehour":"hour", "occurrenceday":"day", "occurrencemonth":"month", "occurrenceyear":"year", "occurrencedayofweek":"day_of_week"}, inplace=True)
df7 = df6.merge(df2, on=["hour", "day", "month", "year", "day_of_week"])
df13 = df2.copy()
df13.rename(columns={'time_id': 'reportedTimeID', "hour":"reportedhour", "day":"reportedday", "month":"reportedmonth", "year":"reportedyear", "day_of_week":"reporteddayofweek"}, inplace=True)
df14 = df13.merge(df7, on=["reportedhour", "reportedday","reportedmonth", "reportedyear", "reporteddayofweek"])
df14 = df14[['event_unique_id','crimeID','locationID','time_id','reportedTimeID', "Long", "Lat","premisetype"]]
print(df14)
df14.to_csv("tableEvent.csv", index=False)

       event_unique_id  crimeID  locationID  time_id  reportedTimeID  \
0       GO-20141756091   100000      300011   600000          600000   
1       GO-20141756091   100000      300171   600000          600000   
2       GO-20141755910   100000      300140   600986          600000   
3       GO-20141755910   100000      300140   600986          600000   
4       GO-20142996126   100241      300140   600001          600001   
...                ...      ...         ...      ...             ...   
405639  GO-20191496247   157000      300085   734808          806309   
405640  GO-20191496247   157000      300274   734808          806309   
405641  GO-20191496261   157000      300126   741119          806309   
405642  GO-20191496261   157000      300126   741119          806309   
405643  GO-20191496218   157000      300126   741912          806309   

             Long        Lat premisetype  
0      -79.450104  43.718735       House  
1      -79.450104  43.718735       House  
2     

In [33]:
df3['Neighbourhood'] = [i.split('(')[0] for i in df3['Neighbourhood']]
df3.to_csv("tableNeighbourhood.csv")
print(df3)

     locationID  Hood_ID                  Neighbourhood Division
0        300000      132                       Malvern       D42
1        300001       76           Bay Street Corridor       D52
2        300002        1        West Humber-Clairville       D23
3        300003       47            Don Valley Village       D33
4        300004       90                 Junction Area       D11
..          ...      ...                            ...      ...
224      404110      109            Caledonia-Fairbank       D12
225      405320        3  Thistletown-Beaumond Heights       D31
226      416423       44               Flemingdon Park       D33
227      421705       75         Church-Yonge Corridor       D12
228      427148       32           Englemount-Lawrence       D42

[229 rows x 4 columns]
